In [1]:
import nltk
#nltk.download()

In [2]:
import requests
import pandas as pd
import re, string, unicodedata
from pprint import pprint
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import FreqDist
from textblob import TextBlob

#import contractions
#import inflect

In [3]:
violence_words = set(["asesinato", "asesino", "asesina", "matar", "mata", "mato", "apuñalar","apuñalo","pega","pego","apalizo","golpea","golpeo","agredio","agrede"])

In [4]:
print(violence_words)

{'matar', 'apuñalar', 'agredio', 'golpea', 'asesino', 'pega', 'pego', 'mata', 'apuñalo', 'apalizo', 'golpeo', 'asesina', 'agrede', 'asesinato', 'mato'}


In [5]:
stopWords = set(stopwords.words('spanish'))

In [6]:
print(stopWords)

{'sean', 'tenidas', 'hemos', 'habré', 'habido', 'fuisteis', 'tendríais', 'no', 'fueses', 'otras', 'vuestra', 'estén', 'estarías', 'tenemos', 'sentid', 'tuyos', 'nuestra', 'ellas', 'tiene', 'del', 'todos', 'poco', 'estuviera', 'con', 'hayamos', 'habríais', 'esta', 'estad', 'ha', 'sintiendo', 'habéis', 'esto', 'desde', 'su', 'esos', 'estar', 'os', 'mío', 'nuestros', 'estuvisteis', 'soy', 'fueseis', 'tengo', 'una', 'será', 'tienen', 'habíamos', 'tenía', 'mis', 'tenéis', 'fuera', 'de', 'habíais', 'hubo', 'tendrán', 'serían', 'tuvieran', 'estuviésemos', 'estaremos', 'tuvieses', 'estamos', 'sentidos', 'porque', 'tenían', 'han', 'seré', 'otros', 'y', 'hubiéramos', 'seas', 'sí', 'tened', 'tuvimos', 'nuestras', 'algunas', 'el', 'la', 'estaríais', 'ellos', 'estaríamos', 'hubimos', 'estemos', 'tuvisteis', 'estuviste', 'seamos', 'eran', 'tengáis', 'vuestras', 'míos', 'tuvo', 'estuviesen', 'me', 'hubieran', 'fuéramos', 'haya', 'tendría', 'fuesen', 'uno', 'fuésemos', 'has', 'hayáis', 'tuvierais', 't

In [7]:
MADRID_URL = "https://www.madridiario.es/sucesos"

In [8]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") #parser para que no se queje
    return soup

soup = get_soup(MADRID_URL)

In [9]:
#Get headers and its publication date
def get_headers(soup):
    headers = []
    date = []
    
    container = soup.find_all('div',{'class','n1 fueraNoticia'})
    for item in container:
        headers.append(item.find('h2',{'class','titulo'}).text)
        date.append(item.find('meta',itemprop='datePublished').get('content')[:10])

    if len(headers) == len(date):
        return pd.DataFrame({"headers": headers, "datePublication": date})
    else:
        return -1 #exit with error because there is news with date

        
dfnews_raw = get_headers(soup)
dfnews_raw.head(11)

,headers,datePublication
0,"La piscina de Peñuelas, cerrada durante toda l...",2019-06-29
1,Rescatan el cuerpo sin vida de una madrileña e...,2019-06-29
2,Más de 200 efectivos de la UME y Bomberos trab...,2019-06-29
3,Detenido en Barcelona un hombre sospechoso de ...,2019-06-29
4,Más de 1.300 hectáreas calcinadas en Madrid po...,2019-06-29
5,"Una anciana, intoxicada al incendiarse su casa...",2019-06-28
6,Apuñalamiento mortal en un parque de Valdezarza,2019-06-28
7,Herida grave al volcar su coche en Los Santos ...,2019-06-28
8,Un joven en silla de ruedas cae a las vías en ...,2019-06-28
9,Desalojado un edificio por un incendio que dej...,2019-06-28


In [10]:
"""
I will try to do all in just one function called clean

for i in range(0,len(df['tweet'])):
			# get rid of anythin that isnt a letter

			exclusion_list = ['[^a-zA-Z]','rt', 'http', 'co', 'RT']
			exclusions = '|'.join(exclusion_list)
			text = re.sub(exclusions, ' ' , df['tweet'][i])
			text = text.lower()
			words = text.split()
			words = [word for word in words if not word in stopword_list]
			 # only use stem of word
			#words = [ps.stem(word) for word in words]
			df['clean_tweets'][i] = ' '.join(words)
"""

"\nI will try to do all in just one function called clean\n\nfor i in range(0,len(df['tweet'])):\n\t\t\t# get rid of anythin that isnt a letter\n\n\t\t\texclusion_list = ['[^a-zA-Z]','rt', 'http', 'co', 'RT']\n\t\t\texclusions = '|'.join(exclusion_list)\n\t\t\ttext = re.sub(exclusions, ' ' , df['tweet'][i])\n\t\t\ttext = text.lower()\n\t\t\twords = text.split()\n\t\t\twords = [word for word in words if not word in stopword_list]\n\t\t\t # only use stem of word\n\t\t\t#words = [ps.stem(word) for word in words]\n\t\t\tdf['clean_tweets'][i] = ' '.join(words)\n"

In [11]:
# Get rid of lower-cases
def tolower (s):
    return s.lower()


dfnews_raw.headers = dfnews_raw.headers.apply(tolower)
dfnews_raw.head(6)

,headers,datePublication
0,"la piscina de peñuelas, cerrada durante toda l...",2019-06-29
1,rescatan el cuerpo sin vida de una madrileña e...,2019-06-29
2,más de 200 efectivos de la ume y bomberos trab...,2019-06-29
3,detenido en barcelona un hombre sospechoso de ...,2019-06-29
4,más de 1.300 hectáreas calcinadas en madrid po...,2019-06-29
5,"una anciana, intoxicada al incendiarse su casa...",2019-06-28


In [12]:
# Get rid of accents
def clean_accent(s):
    return s.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')

dfnews_raw.headers = dfnews_raw.headers.apply(clean_accent)
dfnews_raw.head(6)

,headers,datePublication
0,"la piscina de peñuelas, cerrada durante toda l...",2019-06-29
1,rescatan el cuerpo sin vida de una madrileña e...,2019-06-29
2,mas de 200 efectivos de la ume y bomberos trab...,2019-06-29
3,detenido en barcelona un hombre sospechoso de ...,2019-06-29
4,mas de 1.300 hectareas calcinadas en madrid po...,2019-06-29
5,"una anciana, intoxicada al incendiarse su casa...",2019-06-28


In [13]:
#trying to pass severals parameters without functools and operator
#dfnews_raw.headers = dfnews_raw.headers.apply(clean_any_character(work_list,','))
#dfnews_raw.headers = dfnews_raw.headers.apply(clean_charac, character=',')
#dfnews_raw.headers = dfnews_raw.headers.apply(clean_charac, arg=(dfnews_raw,',')


In [14]:
#Get rid of commas
def clean_commas(s):
    return s.replace(',','')

dfnews_raw.headers = dfnews_raw.headers.apply(clean_commas)
dfnews_raw.head(11)

,headers,datePublication
0,la piscina de peñuelas cerrada durante toda la...,2019-06-29
1,rescatan el cuerpo sin vida de una madrileña e...,2019-06-29
2,mas de 200 efectivos de la ume y bomberos trab...,2019-06-29
3,detenido en barcelona un hombre sospechoso de ...,2019-06-29
4,mas de 1.300 hectareas calcinadas en madrid po...,2019-06-29
5,una anciana intoxicada al incendiarse su casa ...,2019-06-28
6,apuñalamiento mortal en un parque de valdezarza,2019-06-28
7,herida grave al volcar su coche en los santos ...,2019-06-28
8,un joven en silla de ruedas cae a las vias en ...,2019-06-28
9,desalojado un edificio por un incendio que dej...,2019-06-28


In [15]:
#Get rid of quotes
def clean_quotes(s):
    return s.replace('\"','').replace('\'','')

dfnews_raw.headers = dfnews_raw.headers.apply(clean_quotes)

dfnews_clean_string = dfnews_raw.copy() #copy for sentiment study
dfnews_clean_token = dfnews_raw.copy() #copy for word frequency study
dfnews_clean_string.head(18)

,headers,datePublication
0,la piscina de peñuelas cerrada durante toda la...,2019-06-29
1,rescatan el cuerpo sin vida de una madrileña e...,2019-06-29
2,mas de 200 efectivos de la ume y bomberos trab...,2019-06-29
3,detenido en barcelona un hombre sospechoso de ...,2019-06-29
4,mas de 1.300 hectareas calcinadas en madrid po...,2019-06-29
5,una anciana intoxicada al incendiarse su casa ...,2019-06-28
6,apuñalamiento mortal en un parque de valdezarza,2019-06-28
7,herida grave al volcar su coche en los santos ...,2019-06-28
8,un joven en silla de ruedas cae a las vias en ...,2019-06-28
9,desalojado un edificio por un incendio que dej...,2019-06-28


In [16]:
# Get each word / Tokenizar
def token(s):
    return nltk.word_tokenize(s)

dfnews_clean_token.headers = dfnews_clean_string.headers.apply(token)
dfnews_clean_token.head(6)

,headers,datePublication
0,"[la, piscina, de, peñuelas, cerrada, durante, ...",2019-06-29
1,"[rescatan, el, cuerpo, sin, vida, de, una, mad...",2019-06-29
2,"[mas, de, 200, efectivos, de, la, ume, y, bomb...",2019-06-29
3,"[detenido, en, barcelona, un, hombre, sospecho...",2019-06-29
4,"[mas, de, 1.300, hectareas, calcinadas, en, ma...",2019-06-29
5,"[una, anciana, intoxicada, al, incendiarse, su...",2019-06-28


In [17]:
def clean_stopwords(lst):
    
    [row.remove(item) for row in list_tokens for item in row if item in stp_words]
    return list_tokens
    
cleaned_data = clean_stopwords(token_news, stopWords)
pprint(cleaned_data[:6])

NameError: name 'token_news' is not defined

In [ ]:
#new = new + " enfado"
new = [' '.join(item) for item in cleaned_data]


for item in new:
    if TextBlob(item).detect_language() == 'es':
        print("1")
        analysis = TextBlob(str(item))
        item = analysis.translate(to='spanish')
        print(item)
    analysis = TextBlob(str(item))
    print(analysis.sentiment,"\n")




"""
new = ' '.join(cleaned_data[1])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")


new = ' '.join(cleaned_data[2])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")

"""
"""
new2 = analysis.translate(to='spanish')
analysis = TextBlob(str(new2))
print(analysis.sentiment)
print(analysis.tags)
#print(analysis.translate(to='spanish'))
print(dir(analysis))
"""


In [ ]:
def new_item(list_tokens):
    for index, row in enumerate(list_tokens):
        list_tokens.insert(index+1, [])
        list_tokens.insert(index+1, [])
    return list_tokens
 
print(new_item(cleaned_data))

In [ ]:
def violence_at_row(list_tokens,set_list):
    for row in list_tokens:
        for word in row:
            if word in violence_words:
                

In [ ]:
for word in cleaned_data:
    if 